# NRM python upstream client library tutorial

This tutorial covers the use of NRM's python upstream client library, in the context of running an external resource management strategy. The next cell imports the upstream client library. and configures hosts.

In [8]:
from nrm.upstreamclient import Local,Remote

This notebook will start `nrmd` on the same machine as the notebok, but the same interface should be available for remote execution:

In [9]:
host=Local()
#host=Remote( target="cc@129.114.108.201")

Note that the two classes offer the same method to start nrmd and interact via blocking message passing primitives. The following cell should show their docstrings and code when executed interactively.

In [10]:
# Local??
# Remote??

The next cell defines some dode (`nrmd`) daemon configurations as dictionaries:

In [11]:
daemonCfgs = {
    'redirected_log': { "logfile" : "/tmp/logfile_experiment1"},
    #"other":'todo'
}

A workload need a command, some arguments, and a manifest, also represented as a python dictionary.

In [25]:
workloads = {
    "dummy":{ "cmd":"sleep",
              "args": ["333"] ,
              "sliceID": "toto",
              "manifest":{"app": {
                              "slice": {
                                  "cpus": 1,
                                  "mems": 1 } 
                                       },
                          "name": "default"
                         },
            },
    #"other":todo
}

Manifest options are documented in schema file [resources/manifestSchema.json](../../resources/manifestSchema.json) . Clicking this link should open a useful tab - modern web browsers have JSON data explorers. Example manifest files are in [resources/examples](../../resources/examples) in JSON/YAML/Dhall format. For instance, the manifest file [resources/examples/perfwrap.json](../../resources/examples/perfwrap.json) enables enables performance monitoring:

In [26]:
cat ../../resources/examples/perfwrap.json | jq

{
  "hwbind": false,
  "app": {
    "scheduler": {
      "fIFO": {}
    },
    "power": {
      "slowdown": 1,
      "profile": false,
      "policy": "noPowerPolicy"
    },
    "perfwrapper": {
      "perfwrapper": {
        "perfLimit": 100000,
        "perfFreq": 1
      }
    },
    "slice": {
      "cpus": 1,
      "mems": 1
    }
  },
  "name": "default"
}


Under-specified manifests like the one in our `workloads` above (with missing optional fields from the schema) fill missing values with defaults, which can be seen in file [resources/examples/default.json](../../resources/examples/default.json):

In [27]:
cat ../../resources/examples/default.json | jq

{
  "hwbind": false,
  "app": {
    "scheduler": {
      "fIFO": {}
    },
    "power": {
      "slowdown": 1,
      "profile": false,
      "policy": "noPowerPolicy"
    },
    "perfwrapper": {
      "perfwrapperDisabled": {}
    },
    "slice": {
      "cpus": 1,
      "mems": 1
    }
  },
  "name": "default"
}


The next cell defines some experiments:

In [28]:
experiments = { "example": (daemonCfgs['redirected_log'], workloads["dummy"]),
                #"other": todo
              }

In [33]:
host.start_daemon(daemonCfgs['redirected_log'])

In [40]:
host.check_daemon()

False

In [37]:
assert(host.check_daemon())

AssertionError: 

In [41]:
host.stop_daemon()
assert(host.check_daemon() == False)

Exception: 

We now are ready to run an external resource management strategy. Using the low-level message passing interface:

In [29]:
for name, (daemonCfg, workload) in experiments.items():
    #starting the daemon (does act as silent restart)
    host.start_daemon(daemonCfg)
    #running the workload
    workloadHandler = host.run_workload(workload)
    #message passing exchange: 
    while not workloadHandler.finished():
        measurement_message = host.recv()
        #insert your code here
        host.send(command_message)
    print(workloadHandler.exit_status())
    print(host.check_daemon())
    
host.stop_daemon()

AttributeError: 'Local' object has no attribute 'run_workload'